In [1]:
from random import shuffle
from itertools import combinations
import numpy as np


## Sample Documents

In [2]:
docs = [
    "flying fish flew by the space station",
    "we will not allow you to bring your pet armadillo along",
    "he figured a few sticks of dynamite were easier than a fishing pole to catch fish"
]

## Shingling And One-Hot Encoding

In [3]:
def shingle_document(doc, k=2):
    shingles = set()
    for i in range(len(doc) - k + 1):
        shingles.add(doc[i:i+k])
    return shingles

def shingle_documents(docs, k=2):
    return [shingle_document(d, k) for d in docs]

def create_vocab(shingls_list):
    vocab = shingls_list[0]
    for s in shingls_list[1:]:
        vocab = vocab.union(s)
    return list(vocab)

def create_1hot_enconding(vocab, shingles):
    return [1 if v in shingles else 0 for v in vocab]

def create_1hot_encondings(vocab, shingles_list):
    return [create_1hot_enconding(vocab, s) for s in shingles_list]

In [4]:
shingled_docs = shingle_documents(docs)
vocab = create_vocab(shingled_docs)
onehot_docs = create_1hot_encondings(vocab, shingled_docs)

In [5]:
print(onehot_docs[0])

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]


## MinHashing

In [6]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

def create_hash(one_hot_vector: list,minhash_func: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = one_hot_vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

def create_signatures(onehot_docs: list, minhash_func: list):
    # create the signatures for all the documents
    return [create_hash(doc, minhash_func) for doc in onehot_docs]


In [7]:
# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 24)

# we create the signatures for all the documents
signatures = create_signatures(onehot_docs, minhash_func)

In [8]:
for doc, sig in zip(docs, signatures):
    print(f"{doc[:30]}... , {sig}")

flying fish flew by the space ... , [44, 43, 70, 93, 44, 98, 27, 43, 71, 93, 35, 44, 20, 30, 21, 99, 21, 87, 20, 27, 71, 21, 87, 87]
we will not allow you to bring... , [88, 94, 31, 45, 24, 60, 27, 3, 41, 40, 35, 14, 39, 62, 59, 18, 21, 28, 46, 27, 85, 21, 33, 34]
he figured a few sticks of dyn... , [2, 94, 70, 34, 64, 87, 27, 43, 54, 10, 36, 47, 20, 62, 38, 2, 21, 87, 79, 27, 48, 21, 53, 87]


## Evaluation of MinHashing

In [9]:
def jacard_similarity(sig1, sig2):
    # calculate the jacard similarity
    return len(set(sig1).intersection(sig2)) / len(set(sig1).union(sig2))

In [10]:
ind1 , ind2 = 0, 2
print(f"Similarity(raw) between {[ind1]} and {[ind2]} is {jacard_similarity(docs[ind1], docs[ind2])}")
print(f"Similarity(1h) between {[ind1]} and {[ind2]} is {jacard_similarity(onehot_docs[ind1], onehot_docs[ind2])}")
print(f"Similarity(minhash) between {[ind1]} and {[ind2]} is {jacard_similarity(signatures[ind1], signatures[ind2])}")

Similarity(raw) between [0] and [2] is 0.7272727272727273
Similarity(1h) between [0] and [2] is 1.0
Similarity(minhash) between [0] and [2] is 0.23076923076923078


## Locality Sensitive Hashing

In [11]:
def create_band(signature, b: int):
    assert len(signature) % b == 0
    r = len(signature) // b
    subvecs = [signature[i:i+r] for i in range(0, len(signature), r)]
    return subvecs

def create_bands(signatures, b: int):
    return [create_band(sig, b) for sig in signatures]

def is_candidate_match(band1, band2):
    for b1, b2 in zip(band1, band2):
        if b1 == b2:
            print(f"Found a candidate match: {b1} == {b2}")
            return True
    return False
        

In [12]:
bands = create_bands(signatures, 6)

In [13]:
print(bands[0])
print(bands[1])
print(bands[2])

print(is_candidate_match(bands[0], bands[1]))
print(is_candidate_match(bands[0], bands[2]))
print(is_candidate_match(bands[1], bands[2]))

[[44, 43, 70, 93], [44, 98, 27, 43], [71, 93, 35, 44], [20, 30, 21, 99], [21, 87, 20, 27], [71, 21, 87, 87]]
[[88, 94, 31, 45], [24, 60, 27, 3], [41, 40, 35, 14], [39, 62, 59, 18], [21, 28, 46, 27], [85, 21, 33, 34]]
[[2, 94, 70, 34], [64, 87, 27, 43], [54, 10, 36, 47], [20, 62, 38, 2], [21, 87, 79, 27], [48, 21, 53, 87]]
False
False
False


## LSH Class

In [18]:
class LSH:
    def __init__(self, b):
        self.b = b
        self.buckets = []
        self.counter = 0
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature):
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        # break signature into subvectors
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)
    
    def add_hash(self, signature):
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def add_hashes(self, signatures):
        for sig in signatures:
            self.add_hash(sig)

    def check_candidates(self):
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

In [21]:
lsh = LSH(b=12)

In [22]:
lsh.add_hashes(signatures)

In [27]:
print(len(lsh.buckets))

12


In [31]:
lsh.buckets[3]

{'27,43': [0, 2], '27,3': [1]}